<a href="https://colab.research.google.com/github/Iqmahrizky2/CBIR/blob/main/CBIR.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from __future__ import division
import numpy as np
import pandas as pd
from keras.models import Model
from collections import defaultdict


def get_outputs(input_data, model):
    layer_names = [layer.name for layer in model.layers if
                   'flatten' not in layer.name and 'input' not in layer.name
                   and 'predictions' not in layer.name]
    intermediate_layer_model = Model(inputs=model.input,
                                     outputs=[model.get_layer(layer_name).output for layer_name in layer_names])
    layer_outputs = intermediate_layer_model.predict(input_data)
    return layer_outputs, layer_names


def retrieval(query_data, test_data, query_label, test_labels, model):
    '''
    retrieval task
    :param query_data: query input
    :param test_data: test data set
    :param query_label: predicted label of query input
    :param test_labels: test label
    :param model: investigated model
    :return:
        df: dataframe
        f1: F1 measure
    '''
    related = 0
    retrieved_related = 0
    retrieved = 0
    img_index = []
    img_label = []
    similarity = []
    query_outputs = get_outputs(query_data, model)[0]
    query_feat = query_outputs[5][0]

    layer_outputs = get_outputs(test_data, model)[0]
    layer_output = layer_outputs[5]

    for i in xrange(10000):
        test_feat = layer_output[i]
        test_label = np.argmax(test_labels[i])
        if query_label == test_label:
            related += 1
        # use cosine distance as similarity metric
        cos = np.dot(query_feat, test_feat) / (np.linalg.norm(query_feat)*np.linalg.norm(test_feat))
        sim = 0.5 + 0.5*cos
        if sim >= 0.85:
            retrieved += 1
            img_index.append(i)
            similarity.append(sim)
            img_label.append(test_label)
            if query_label == test_label:
                retrieved_related += 1

    # evaluation metric
    recall = retrieved_related / related
    precision = retrieved_related / retrieved
    f1 = recall * precision * 2 / (recall + precision)

    df = pd.DataFrame({'retrieved_index': img_index, 'similarity': similarity, 'label': img_label})
    df = df.sort_values('similarity', ascending=True)
    return df, f1

In [ ]:

from __future__ import print_function
from keras.datasets import mnist
from keras.layers import Convolution2D, MaxPooling2D, Input, Dense, Activation, Flatten
from keras.models import Model
from tensorflow.keras.utils import to_categorical



def lenet5(input_tensor=None, train=False):
    nb_classes = 10
    # convolution kernel size
    kernel_size = (5, 5)

    if train:
        batch_size = 256
        nb_epoch = 10

        # input image dimensions
        img_rows, img_cols = 28, 28

        # the data, shuffled and split between train and test sets
        (x_train, y_train), (x_test, y_test) = mnist.load_data()

        x_train = x_train.reshape(x_train.shape[0], img_rows, img_cols, 1)
        x_test = x_test.reshape(x_test.shape[0], img_rows, img_cols, 1)
        input_shape = (img_rows, img_cols, 1)

        x_train = x_train.astype('float32')
        x_test = x_test.astype('float32')
        x_train /= 255
        x_test /= 255

        # convert class vectors to binary class matrices
        y_train = to_categorical(y_train, nb_classes)
        y_test = to_categorical(y_test, nb_classes)

        input_tensor = Input(shape=input_shape)
    elif input_tensor is None:
        print(bcolors.FAIL + 'you have to proved input_tensor when testing')
        exit()

    # block1
    x = Convolution2D(6, kernel_size, activation='relu', padding='same', name='block1_conv1')(input_tensor)
    x = MaxPooling2D(pool_size=(2, 2), name='block1_pool1')(x)

    # block2
    x = Convolution2D(16, kernel_size, activation='relu', padding='same', name='block2_conv1')(x)
    x = MaxPooling2D(pool_size=(2, 2), name='block2_pool1')(x)

    x = Flatten(name='flatten')(x)
    x = Dense(120, activation='relu', name='fc1')(x)
    x = Dense(84, activation='relu', name='fc2')(x)
    x = Dense(nb_classes, name='before_softmax')(x)
    x = Activation('softmax', name='predictions')(x)

    model = Model(input_tensor, x)

    if train:
        # compiling
        model.compile(loss='categorical_crossentropy', optimizer='adadelta', metrics=['accuracy'])

        # trainig
        model.fit(x_train, y_train, validation_data=(x_test, y_test), batch_size=batch_size, epochs=nb_epoch, verbose=1)
        # save model
        model.save_weights('./lenet5.h5')
        score = model.evaluate(x_test, y_test, verbose=0)
        print('\n')
        print('Overall Test score:', score[0])
        print('Overall Test accuracy:', score[1])
    else:
        model.load_weights('./lenet5.h5')
        print(bcolors.OKBLUE + 'Model loaded' + bcolors.ENDC)

    return model


if __name__ == '__main__':
    lenet5(train=True)

In [ ]:
'''
image transformation
'''

import numpy as np
import random
from skimage import draw
from skimage import morphology as mp
from skimage import exposure as ep
from keras.preprocessing import image


def deprocess_img(x):
    # convert tensor to a valid image
    x *= 255
    x = np.clip(x, 0, 255).astype('uint8')
    return x.reshape(x.shape[1], x.shape[2])


def light(img):
    # adjust_gamma(x, gamma)
    # gamma > 1 --- darker
    # gamma < 1 --- brighter
    img = img.astype(np.float32)
    return ep.adjust_gamma(img, random.uniform(0.7, 1.05))


def shift(img):
    # random_shift(x, wrg, hrg, row_axis=1, col_axis=2, channel_axis=0, fill_mode='nearest', cval=0.0)
    img = img.reshape(1, 28, 28)
    wrg = 0.2
    hrg = 0.2
    return image.random_shift(img, wrg, hrg).reshape(28, 28)


def rotate(img):
    # random_rotation(x, rg, row_axis=1, col_axis=2, channel_axis=0, fill_mode='nearest', cval=0.0)
    img = img.reshape(1, 28, 28)
    rg = 30
    return image.random_rotation(img, rg).reshape(28, 28)


def flip(img):
    # flip_axis(x, axis)
    # horizontal
    return image.flip_axis(img, 1)


def zoom(img):
    # random_zoom(x, zoom_range, row_axis=1, col_axis=2, channel_axis=0, fill_mode='nearest', cval=0.0)
    img = img.reshape(1, 28, 28)
    zoom_range = (0.6, 1.3)
    return image.random_zoom(img, zoom_range).reshape(28, 28)


def dilation(img):
    # return greyscale morphological dilation of an image
    return mp.dilation(img, mp.square(2, dtype=np.uint8))


def erosion(img):
    # return greyscale morphological erosion of an image
    return mp.erosion(img, mp.square(2, dtype=np.uint8))


def draw_line(img):
    img = img.reshape(28, 28)
    # draw a straight line across the digit
    r0 = random.randint(1, 27)
    c0 = random.randint(1, 4)
    r1 = random.randint(1, 27)
    c1 = random.randint(24, 27)
    rr, cc = draw.line(r0, c0, r1, c1)
    img[rr, cc] = 255
    return img


def s_p_noise(img):
    # salt and pepper noise
    img = img.reshape(28, 28)
    rows, cols = img.shape
    for i in range(50):
        x = np.random.randint(0, rows)
        y = np.random.randint(0, cols)
        img[x, y] = 255
        x_ = np.random.randint(4, 24)
        y_ = np.random.randint(4, 24)
        img[x_, y_] = 0
    return img